## Jonathan Pedoeem
## Prof. Curro HW4 CIFAR100
## October 4th, 2018

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tqdm import tqdm
from time import time

In [0]:
np.random.seed(1234)
tf.set_random_seed(1234)
IMAGE_HEIGHT = 32
IMAGE_WIDTH  = 32
IMAGE_DEPTH = 3
batch_size = 20
NUM_CLASSES = 100
l2_lamb = 1e-3
lr = 1e-1

In [0]:
def reshape(x):
    return np.reshape(x, [-1,IMAGE_WIDTH,IMAGE_HEIGHT,IMAGE_DEPTH])

In [0]:
def get_cifar100():
    datum = {}
    cifar10 = tf.keras.datasets.cifar100
    (x_train, y_train),(x_test, y_test) = cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    randos = np.random.choice(50000,50000, replace=False)
    validation_randos = randos[:10000]
    train_randos = randos[10000:]
    val_x, val_y = reshape(x_train[validation_randos]), y_train[validation_randos]
    x_train, y_train = reshape(x_train[train_randos]), y_train[train_randos]
    x_test = reshape(x_test)
    y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
    y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)
    val_y = keras.utils.to_categorical(val_y,NUM_CLASSES)
    datum = {"x_train":x_train, "y_train":y_train, "x_val":val_x, "y_val":val_y,"x_test":x_test,"y_test":y_test}
    return datum

In [5]:
datum = get_cifar100()

169009152/169001437 [==============================] - 474s 3us/step


In [0]:
# from keras website
datagen = keras.preprocessing.image.ImageDataGenerator(
    horizontal_flip=True)
test_datagen = keras.preprocessing.image.ImageDataGenerator()
datagen.fit(datum["x_train"])


In [0]:
model = keras.Sequential()
model.add(keras.layers.BatchNormalization(input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH)))
model.add(keras.layers.Conv2D(filters=99,kernel_size=(3,3),input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH), strides=2,padding='valid',activation='relu', kernel_initializer=keras.initializers.VarianceScaling()))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Conv2D(filters=99,kernel_size=(3,3),strides=2,padding='valid',activation='relu', kernel_initializer=keras.initializers.VarianceScaling()))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Conv2D(filters=211,kernel_size=(3,3),strides=2,padding='valid',activation='relu', kernel_initializer=keras.initializers.VarianceScaling()))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Conv2D(filters=211,kernel_size=(3,3),strides=2,padding='same',activation='relu', kernel_initializer=keras.initializers.VarianceScaling()))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(NUM_CLASSES,activation="softmax", kernel_initializer=keras.initializers.VarianceScaling()))
model.compile(optimizer=keras.optimizers.SGD(lr, momentum=0.7),
              loss='categorical_crossentropy',
              metrics=['accuracy', 'top_k_categorical_accuracy'])

In [0]:
model.fit_generator(datagen.flow(datum["x_train"],datum["y_train"], batch_size=512), epochs=200,
          validation_data=(datum["x_val"], datum["y_val"]), verbose=2)

In [0]:
model.compile(optimizer=keras.optimizers.SGD(1e-2, momentum=0.7),
              loss='categorical_crossentropy',
              metrics=['accuracy','top_k_categorical_accuracy'])
model.fit_generator(datagen.flow(datum["x_train"],datum["y_train"], batch_size=256), epochs=200,
          validation_data=(datum["x_val"], datum["y_val"]), verbose=2)

In [12]:
model.evaluate(datum["x_test"],datum["y_test"])

10000/10000 [==============================] - 2s 215us/step


[1.6003288402557374, 0.5663, 0.836]

Top 5 Accuacy: 83.6%
<br>

Top 1 Accuracy: 56.6%